<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


## Objectives


In this block we will try to solve the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, we should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [50]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [51]:
import folium
import pandas as pd

In [52]:
# Import plugins
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates.

The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [53]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [54]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give we any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [55]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

Let's display our space stations on the map to understand where they are located and near which infrastructure facilities.

In [68]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
label = ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E']
coordinate = [[launch_sites_df['Lat'][_], launch_sites_df['Long'][_]] for _ in range(len(launch_sites_df))]

for _ in range(len(launch_sites_df)):
    circle = folium.Circle(coordinate[_], radius=200, color='#d35400', fill=True).add_child(folium.Popup(label[_]))
    marker = folium.map.Marker(coordinate[_], 
                               icon=DivIcon(icon_size=(20,20), 
                                            icon_anchor=(0,0), 
                                            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label[_],
                                           )
                              )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

Now we see that 3 launch sites are located on the coast in Florida and one in California.

# Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not.

In [71]:
spacex_df.sample(5)

,Launch Site,Lat,Long,class,marker_color
53,CCAFS SLC-40,28.563197,-80.576820,0,red
16,CCAFS LC-40,28.562302,-80.577356,0,red
24,CCAFS LC-40,28.562302,-80.577356,1,green
43,KSC LC-39A,28.573255,-80.646895,0,red
35,VAFB SLC-4E,34.632834,-120.610745,0,red


Next, let's create markers for all launch records. If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`.

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object.

In [60]:
marker_cluster = MarkerCluster()

Let's create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value.

In [61]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.sample()

,Launch Site,Lat,Long,class,marker_color
51,CCAFS SLC-40,28.563197,-80.57682,0,red
52,CCAFS SLC-40,28.563197,-80.57682,0,red
53,CCAFS SLC-40,28.563197,-80.57682,0,red
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


Next, for each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`.

In [62]:
site_map.add_child(marker_cluster) #Add marker_cluster to current site_map

# for each row in spacex_df dataframe create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']

for index, row in spacex_df.iterrows():
    coordinate = [row["Lat"], row["Long"]]
    launchSite = row["Launch Site"]
    
    marker = folium.map.Marker(coordinate, 
                               icon=folium.Icon(color='white', 
                                                icon_color=row['marker_color'],
                                               )
                              )
    marker_cluster.add_child(marker)
    
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, we can easily identify which launch sites have relatively high success rates.

# TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a `Mouse Position` on the map to get coordinate for a mouse over a point on the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interests (such as railway).

In [14]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(position='topright', 
                               separator='; Long: ', 
                               empty_string='NaN', 
                               lng_first=False, 
                               num_digits=20, 
                               prefix='Lat: ', 
                               lat_formatter=formatter, 
                               lng_formatter=formatter,
                              )

site_map.add_child(mouse_position)

Now we can easily find the distance from sites to infrastructure facilities (railway, highway, coastline, etc.) by zooming in and hovering the mouse cursor.

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Let's mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [76]:
launch_site_lat, launch_site_lon = launch_sites_df.iloc[2]["Lat"], launch_sites_df.iloc[2]["Long"]
lat_coastline, long_coastline = 28.56297, -80.56789
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, lat_coastline, long_coastline)

print('Site coordinates:      {}, {}'.format(launch_site_lat, launch_site_lon))
print('Coastline coordinates: {}, {}'.format(lat_coastline, long_coastline))
print('Distance in km:       ', distance_coastline)

Site coordinates:      28.57325457, -80.64689529
Coastline coordinates: 28.56297, -80.56789
Distance in km:        7.802149188552683


In [17]:
# Let's create and add a `folium.Marker` on we selected closest coastline point on the map 
#display the distance between coastline point and launch site using the icon property
coordinate = [lat_coastline, long_coastline]

distance_coastline_marker = folium.Marker(coordinate, 
                                          icon=DivIcon(icon_size=(20,20), 
                                                       icon_anchor=(0,0), 
                                                       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline)))

site_map.add_child(distance_coastline_marker)

In [18]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
points = []
points.append([launch_site_lat, launch_site_lon])
points.append([lat_coastline, long_coastline])
lines=folium.PolyLine(locations=points, weight=1)
site_map.add_child(lines)

Similarly, we can draw a line between a launch site to its closest city, railway, highway, etc. We need to use `MousePosition` to find the their coordinates on the map first.

Infrastructure facilities in Florida:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</left>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</left>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</left>

<center>
    <img src="IBM-DS0321EN-SkillsNetwork/labs/module_3/1.jpg">

Infrastructure facilities in California:

In [85]:
%%html <img src="1.jpg" />

UsageError: %%html is a cell magic, but the cell body is empty.


A city map symbol may look like this:


Now we see that the stations are located in close proximity to the coastlines, and the transport infrastructure near the stations is developed. The nearest major cities are located at a comparative distance from the stations.

## Next Steps:
Next, we will build a dashboard using Ploty Dash on detailed launch records.

## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2022-01-01)


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
